# Da3n3rys Playbook 01 - Memory Dump Analisys for Windows OS

This playbook executes Volatility and other tools in order to extract the most common and useful information from a memory image. 
<a id='table_of_contents'></a>
### Table of contents

- [1. Image Identification](#image_identification)
- [2. Users Information & Activity](#users_information)
- [3. Registry Information](#registry_information)
- [4. Console History](#console_history)
- [5. Users Credentials](#users_credentials)
- [6. Browser History](#browser_history)
- [7. Process & Services Information](#process_information)
- [8. Files Information](#files_information)
- [9. Network Information](#network_information)
- [10. Windows Event Logs Extraction](#windows_events)
- [11. Malicious Code Investigation](#malicious_code)
- [12. Advanced Investigation - Extracting Files](#advanced_extract_files)
- [13. Advanced Investigation - Searching Strings](#advanced_search_strings)
- [14. IOCs List](#iocs_list)

### Prerequisites

Please, in order to execute this playbook, complete or modify these information in the following section:
    
    base_path: Full path of the DFIR Playbooks.
    case_path: Full path for the incident case. This folder will host all the reports extracted from the analysis.
    dump_path: Full path for the extracted files from the case.
    filename: Name of the file to be analyzed. Place the file in the case_path.

And also, install the following dependencies:

    yarascan
    dot - sudo apt install graphviz

This playbook also works with <b>hiberfil.sys</b> and <b>pagefile.sys</b> files and <b>.vmem</b>(Virtual Machine Memory) files. If you want to analyze these files, please, don't change it's names or extensions.

In [ ]:
base_path="/home/Notebooks"
case_path="/home/Notebooks/evidences/case2"
dump_path="/home/Notebooks"
plugins_path="/home/Notebooks/tools/volatility/contrib/plugins"
css_file="/home/Notebooks/css/custom.css"
filename="Virgo.dmp"

from IPython.display import display, HTML, Image

#HTML(open(css_file, "r").read())

<a id='image_identification'></a>
### 1. Image Identification
[Top](#table_of_contents)

<b>md5 image hash</b>

In [ ]:
md5=!md5sum $case_path/$filename
print(md5)

<b>imageinfo:</b> Identifies the operating system, service pack, and hardware architecture (32 or 64 bit), but it also contains other useful information such as the DTB address* and time the sample was collected.

<b>WARNING: This module will select an image profile automatically, please, change manually the profile variable if you need to use another one.</b>

*DTB (Directory Table Base) is what Volatility uses to translate virtual addresses to physical addresses. 

In [ ]:
imageinfo=!python $base_path/tools/volatility/vol.py -f $case_path/$filename imageinfo

for row in imageinfo[2:]:
    if "Suggested" in row:
        print(row)
        profile = row.split()[3].replace(',', '')
       
#Uncomment if needed
profile="Win10x64_15063"
        
print("Profile selected for this analysis: ", profile)

<b>kdbgscan:</b> As opposed to imageinfo which simply provides profile suggestions, kdbgscan is designed to positively identify the correct profile and the correct KDBG address (if there happen to be multiple). This plugin scans for the KDBGHeader signatures linked to Volatility profiles and applies sanity checks to reduce false positives. 

If you find 0 processes and 0 modules in this output and another KDBG with more data, copy its Offset value here:

kdgb="--kdbg=Offset (V)"

In [ ]:
kdgb=""
!python $base_path/tools/volatility/vol.py -f $case_path/$filename kdbgscan

<b>hiberfil.sys:</b> Since the hibernation file is just a compressed copy of the memory, we can use Volatility the same way we would with any other memory sample. But, we will need to first decompress it by converting it to a raw memory dump. This too can be done using Volatility. 

Volatility will actually work with the hiberfil.sys file, but it is extremely slow because it will have to decompress it every time a different plugin is used. Just decompress it once using the imagecopy plugin and it will speed up the process.

In [ ]:
if "hiberfil.sys" in filename:
    !python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile imagecopy -O $case_path/rawimage.img
    filename="rawimage.img"

<b>pagefile.sys:</b> Microsoft Windows uses a paging file, called pagefile.sys, to store page-size blocks of memory that do not current fit into physical memory.
This file, stored in %SystemDrive%\pagefile.sys is a hidden system file and it can never be read or accessed by a user, including Administrator.

It is possible to read this file by parsing the raw file system, or exact it using tools like FTKImager.
Contrary to hybernation files, page files cannot be processed with Volatility: in fact the page file is just the “holes” in memory where blocks are stored to disk, it will often contain information that can be relevant to the case you are trying to solve.

Because storage locations in the paging file are not necessarily sequential, it is unlikely to find consecutive pages there.
Although it is possible to find data in chunks smaller than or equal to 4KB, its the largest an examiner can hope for.
So, the most productive method for analyzing paging files is searching for strings.

In [ ]:
if "pagefile.sys" in filename:
    print("List all paths in pagefile")
    !strings pagefile.sys | grep -i "^[a-z]:\\\\" | sort | uniq | less 
    print("Search for enviroment variables")
    !strings pagefile.sys | grep -i "^[a-zA-Z09_]*=.*" | sort -u | uniq | less
    print("Search for URLs:")
    !strings pagefile.sys | egrep "^https?://" | sort | uniq | less
    print("Search for email addresses:")
    !strings pagefile.sys | egrep '([[:alnum:]_.-]{1,64}+@[[:alnum:]_.-]{2,255}+?\.[[:alpha:].]{2,4})' 
    print("Search malware using yararules")
    !yara malware_rules.yar pagefile.sys
    #stops the notebook execution
    %%javascript
    Jupyter.notebook.session.delete();

<b>.vmem:</b> Virtual Memory or VMEM is a file extension used by virtualization software like VMware, Virtualbox etc. VMEM files generally store RAM or Physical Memory of the related virtual machine.

In [ ]:
if ".vmem" in filename:
    !python $base_path/tools/volatility/vol.py -f $case_path/$filename imagecopy -O $case_path/rawimage.img --profile=$profile
    filename="rawimage.img"

<a id='users_information'></a>
### 2. Users Information & Activity
[Top](#table_of_contents)

<b>clipboard:</b> Extract the contents of the windows clipboard.

In [ ]:
!python /home/mmatari/Notebooks/tools/volatility/vol.py -f $case_path/$filename --profile=$profile clipboard $kdgb

<b>wndscan:</b> This command scans for tagWINDOWSTATION objects and prints details on the window station*, its global atom table, available clipboard formats, and processes or threads currently interacting with the clipboard.

From a forensic standpoint, by analyzing window stations objects, you can detect applications snooping on clipboard activity along with the frequency of clipboard usage and the available data formats.

If the <b>cNumClipFormats</b> and <b>iClipSerialNumber</b> are both 0, this means this window station’s clipboard has never been used.

Windows station act as security boundaries for processes and desktops. If you're not already familiar with these objects, see Sessions, Desktops, and Window Stations (http://blogs.technet.com/b/askperf/archive/2007/07/24/sessions-desktops-and-windows-stations.aspx ) or Window Stations and Desktops (https://docs.microsoft.com/es-es/windows/win32/winstation/window-stations-and-desktops ).

More information about Clipboard Snooping Malware here: https://volatility-labs.blogspot.com/2012/09/movp-12-window-stations-and-clipboard.html

In [ ]:
!python /home/mmatari/Notebooks//tools/volatility/vol.py -f $case_path/$filename --profile=$profile wndscan $kdgb

<b>deskscan:</b> This command subclasses the wndscan plugin and for each window station found, it walks the list of desktops. It can be used for the following purposes:

   - Find rogue desktops used to hide applications from logged-on users.
   - Detect desktops created by ransomware.
   - Link threads to their desktops.
   - Analyze the desktop heap for memory corruptions.
   - Profile dekstop heap allocations to locate USER objects.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile deskscan $kdgb

<b>notepad:</b> List currently displayed notepad text.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile notepad $kdgb

<b>Recently clicked applications and shortcuts:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile userassist $kdgb | grep REG_BINARY

<b>Find prefetch files:</b> Recently executed programs.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile mftparser $kdgb | grep \.pf$

<b>sessions:</b> A plugin to report on active (and in some cases, terminated) logon sessions from Windows based memory dumps, including their associated processes, kernel modules, pool, and page tables. 

Here are some ways you might use this plugin:

   - <b>Link processes to their logon sessions:</b> you see abc.exe running and you want to know if it was launched by someone over RDP or by the user physically sitting at the console. 
   - <b>Detect hidden processes:</b> each session structure contains a linked list of processes for that session. If malware unlinks a process from PsActiveProcessHead, you can leverage this alternate process listing as a means to identify the hidden process. 
   - <b>Determine kernel drivers:</b> each session structure contains a list of drivers mapped into the session. You can use this to distinguish RDP sessions from console or fast-user switching sessions and also for miscellaneous tasks like determining if the system is a VMware virtual machine. 


In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile sessions $kdgb

<b>Timeline:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile timeliner $kdgb

In [ ]:
!python $base_path/tools/autotimeliner/autotimeline.py -f $case_path/$filename -p $profile

<b>Windows 10 Timeline Database:</b> Windows 10 provides a new database called Timeline that, if active, can give us information regarding the tasks performed by all the users in the OS.
More information of this database, here:
https://onlinelibrary.wiley.com/doi/epdf/10.1111/1556-4029.13875?author_access_token=inv4iYhlWdYKUupNglpSQ4ta6bR2k8jH0KrdpFOxC660wwTqHHaN9tPSRNIR_TQPUJ9J1SxuE3LbbtvVI3D6ZaR6V-IDZBOC-gZ1JXjKpSrvltqQD0WUyBOKDafXQka0XhDvuYd_TMuyUFNNSZy0lg%3D%3D&

In [ ]:
#offset=""
#files=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --dump-dir=$dump_path dumpfiles 
#for file in files:
#    if "ActivitiesCache.db" in file:
#        file=file.split()
#        offset=file[0]
#        !python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --offset=$offset --dump-dir=$dump_path dumpfiles 
## download ActivitiesCache.db
## parser https://tzworks.net/prototype_page.php?proto_id=41
## pip install git+https://github.com/forensicmatt/ActivitiesCacheParser
# python3 /usr/local/bin/winactivities2json.py -s $dump_path/ActivitiesCache.db -t $dump_path --debug ERROR -o "{_user}: {StartTime} - {AppId[0][application]}"

<a id='registry_information'></a>
### 3. Registry Information
[Top](#table_of_contents)

<b>hivelist:</b> To locate the virtual addresses of registry hives in memory, and the full paths to the corresponding hive on disk. 

A hive is a logical group of keys, subkeys, and values in the registry that has a set of supporting files containing backups of its data. Each time a new user logs on to a computer, a new hive is created for that user with a separate file for the user profile. This is called the user profile hive.

In [ ]:
hive_list=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile hivelist $kdgb
for line in hive_list[0:3]:
    print(line)
for line in hive_list[3:]:
    print(line)
    line=line.split()
    offset=line[1]
    print(line[1])
    #!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile hivedump --hive-offset $offset

<b>shellbags:</b> This plugin parses and prints Shellbag* information obtained from the registry.

*Shellbag: Built into Microsoft Windows is the ability for the operating system to track user window viewing preferences. These are settings specific to the default Windows shell, WindowsExplorer, such as folder window size, position on the screenand others. They allow consistent user experiences betweensessions

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile shellbags $kdgb

<b>shimcache:</b> Parses the Application Compatibility Shim Cache registry key. This artifact can provide a timeline of which program was executed and when it was first run and last modified

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile shimcache $kdgb

<a id='console_history'></a>
### 4. Console History
[Top](#table_of_contents)

<b>consoles:</b> Extract command history by scanning for \_CONSOLE\_INFORMATION.

In [ ]:
!python /home/mmatari/Notebooks/tools/volatility/vol.py -f $case_path/$filename --profile=$profile consoles $kdgb

<b>cmdscan:</b> Extract command history by scanning for \_COMMAND\_HISTORY.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile cmdscan $kdgb

<b>cmdline:</b> Display process command-line arguments.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile  cmdline $kdgb

<b>Command line search bruteforcing:</b> "/C:\\\\.+>/"

In [ ]:
!egrep -ai "/C:\/.+\>" $case_path/$filename

<b>Command line search bruteforcing:</b>

In [ ]:
!egrep -ai "kitty" $case_path/$filename

<a id='users_credentials'></a>
### 5. Users credentials
[Top](#table_of_contents)

<b>hashdump:</b> Extracts and decrypts cached domain credentials stored in the registry.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile hashdump $kdgb

<b>cachedump:</b> Dumps cached domain hashes from memory.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile cachedump $kdgb

<b>lsadump:</b> To dump LSA secrets from the registry, use the lsadump command. This exposes information such as the default password (for systems with autologin enabled), the RDP public key, and credentials used by DPAPI.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile lsadump $kdgb

<b>mimikatz:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/mimikatz -f $case_path/$filename --profile=$profile mimikatz $kdgb

<b>openvpn:</b> Extracts openvpn credentials.

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/openvpn -f $case_path/$filename --profile=$profile openvpn $kdgb

<a id='browser_history'></a>
### 6. Browser History
[Top](#table_of_contents)

<b>iehistory:</b> Internet Explorer History.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile iehistory $kdgb

<b>firefoxhistory:</b> firefoxhistory, firefoxcookies, and firefoxdownloads plugins to extract the following firefox history data: moz_places, moz_cookies, and moz_downloads.

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile firefoxhistory $kdgb

<b>firefoxcookies:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile firefoxcookies $kdgb

<b>firefoxdownloads:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile firefoxdownloads $kdgb

<b>chromehistory:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile chromehistory $kdgb

<b>chromevisits:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile chromevisits $kdgb

<b>chromesearchterms:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile  chromesearchterms $kdgb

<b>chromedownloads:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile chromedownloads $kdgb 

<b>chromedownloadchains:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile chromedownloadchains $kdgb

<b>chromecookies:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/DaveLasalle -f $case_path/$filename --profile=$profile chromecookies $kdgb

<a id='process_information'></a>
### 7. Process & Services Information
[Top](#table_of_contents)

<b>svcscan:</b> Scan for Windows services.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile svcscan $kdgb

<b>getservicesids:</b> The getservicesids command calculates the SIDs for services on a machine and outputs them in Python dictionary format for future use. The service names are taken from the registry ("SYSTEM\CurrentControlSet\Services").

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile getservicesids $kdgb

<b>getsids:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile getsids $kdgb

<b>psscan:</b> This can find processes that previously terminated (inactive) and processes that have been hidden or unlinked by a rootkit. The downside is that rootkits can still hide by overwriting the pool tag values (though not commonly seen in the wild).

In [ ]:
from IPython.display import Image
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile psscan $kdgb
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile psscan --output=dot --output-file=$dump_path/psscan.dot $kdgb
!dot -Tpng $dump_path/psscan.dot -o $dump_path/psscan.png
Image(dump_path + "/psscan.png")

<b>pslist:</b> This walks the doubly-linked list pointed to by PsActiveProcessHead and shows the offset, process name, process ID, the parent process ID, number of threads, number of handles, and date/time when the process started and exited. As of 2.1 it also shows the Session ID and if the process is a Wow64 process (it uses a 32 bit address space on a 64 bit kernel). This plugin does not detect hidden or unlinked processes (but psscan can do that).



In [ ]:
from IPython.display import Image
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile pslist -P $kdgb
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile pslist -P --output=dot --output-file=$dump_path/pslist.dot $kdgb
!dot -Tpng $dump_path/pslist.dot -o $dump_path/pslist.png
Image(dump_path + "/pslist.png")

<b>psxview:</b> Find hidden processes with various process listings.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile psxview -P $kdgb

<b>envars:</b> Typically this will show the number of CPUs installed and the hardware architecture (though the kdbgscan output is a much more reliable source), the process's current directory, temporary directory, session name, computer name, user name, and various other interesting artifacts.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile envars $kdgb

<b>envars:</b> Find binaries loaded from temporary directories.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile envars PID $kdgb | grep -i "TEMP\|\|TMP"

<b>vadtree:</b> The Virtual Address Descriptor tree is used by the Windowsmemory manager to describe memory ranges used by a process as they are allocated. When a process allocatesmemory with VirutalAlloc, the memory manager creates anentry in the VAD tree. The corresponding page directory andpage table entries are not created until the process tries to ref-erence that memory page, which can provide significantmemory savings for processes that allocate a large amountof memory but access it sparsely (Russinovich and Solomon,2004, pp. 448–449). The tree itself is a self-balancing binarytree: at any given node, memory addresses lower than thosecontained at the current node can be found in the left subtreeand higher ranges in the right.

More information on the VAD:

https://www.dfrws.org/sites/default/files/session-files/paper-the_vad_tree_-_a_process-eye_view_of_physical_memory.pdf

Fillcolor Legend:

    Red: Heaps
    Gray: DLLs
    Green: Stacks
    Yellow: Mapped Files

In [ ]:
from ipywidgets import widgets
from IPython.display import Image,clear_output

text = widgets.Text(description="ProcessID:", width=200)
display(text)

def handle_submit(sender):
    clear_output()
    display(text)
    processid=text.value
    processidimage=processid + ".png"
    processiddot=processid + ".dot"
    !python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile -p $processid vadtree --output=dot --output-file=$dump_path/$processiddot $kdgb
    !dot -Tpng $dump_path/$processiddot -o $dump_path/$processidimage
    display(Image(filename=dump_path +"/"+ processidimage))
        
text.on_submit(handle_submit)

<a id='files_information'></a>
### 8. Files Information
[Top](#table_of_contents)

<b>filescan:</b> To find FILE_OBJECTs in physical memory using pool tag scanning, use the filescan command. This will find open files even if a rootkit is hiding the files on disk and if the rootkit hooks some API functions to hide the open handles on a live system. The output shows the physical offset of the FILE_OBJECT, file name, number of pointers to the object, number of handles to the object, and the effective permissions granted to the object.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile filescan $kdgb

<a id='network_information'></a>
### 9. Network Information
[Top](#table_of_contents)

<b>Network shares information:</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile handles -t File $kdgb | egrep "\\Device\\(LanmanRedirector|Mup)"

<b>sockscan:</b> To find \_ADDRESS_OBJECT structures using pool tag scanning, use the sockscan command. As with connscan, this can pick up residual data and artifacts from previous sockets.

This command is for x86 and x64 Windows XP and Windows 2003 Server only.

In [ ]:
ips_list=[]

sockscan=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile sockscan $kdgb
for row in sockscan[1:]:
    print(row)
    
for row in sockscan[3:]:
    row=row.split()
    ips_list.append(row[5])

<b>sockets:</b> To detect listening sockets for any protocol (TCP, UDP, RAW, etc), use the sockets command. This walks a singly-linked list of socket structures which is pointed to by a non-exported symbol in the tcpip.sys module.

This command is for x86 and x64 Windows XP and Windows 2003 Server only.

In [ ]:
sockets=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile sockets $kdgb
for row in sockets[1:]:
    print(row)

for row in sockets[3:]:
    row=row.split()
    ips_list.append(row[5])

<b>connections:</b> To view TCP connections that were active at the time of the memory acquisition, use the connections command. This walks the singly-linked list of connection structures pointed to by a non-exported symbol in the tcpip.sys module.

This command is for x86 and x64 Windows XP and Windows 2003 Server only.

In [ ]:
connections=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile connections $kdgb

for row in connections[1:]:
    print(row)
    
for row in connections[3:]:
    row=row.split()
    ip_port=row[2]
    ip_port=ip_port.split(':')
    ips_list.append(ip_port[0])

<b>connscan:</b> To find \_TCPT_OBJECT structures using pool tag scanning, use the connscan command. This can find artifacts from previous connections that have since been terminated, in addition to the active ones. In the output below, you'll notice some fields have been partially overwritten, but some of the information is still accurate. For example, the very last entry's Pid field is 0, but all other fields are still in tact. Thus, while it may find false positives sometimes, you also get the benefit of detecting as much information as possible.

This command is for x86 and x64 Windows XP and Windows 2003 Server only.

In [ ]:
connscan=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile connscan $kdgb

for row in connscan[1:]:
    print(row)
    
for row in connscan[3:]:
    row=row.split()
    ip_port=row[2]
    ip_port=ip_port.split(':')
    ips_list.append(ip_port[0])

<b>netscan:</b> To scan for network artifacts in 32- and 64-bit Windows Vista, Windows 2008 Server and Windows 7 memory dumps, use the netscan command. This finds TCP endpoints, TCP listeners, UDP endpoints, and UDP listeners. It distinguishes between IPv4 and IPv6, prints the local and remote IP (if applicable), the local and remote port (if applicable), the time when the socket was bound or when the connection was established, and the current state (for TCP connections only). 

In [ ]:
netscan=!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile netscan $kdgb
for row in netscan[1:]:
    print(row)
    
for row in netscan[3:]:
    row=row.split()
    ip_port=row[2]
    ip_port=ip_port.split(':')
    ips_list.append(ip_port[0])

<b>NDISPktScan:</b> An Ethernet packet, or, depending on the version of Windows, the start of an Ethernet packet, can be found by searching kernel memory space for a source MAC address followed by an EtherType of IPv4 or IPv6. As long as the source MAC address is known, the packets can be found and recovered.

Ideally, the source MAC address will be passed to the plugin via the --mac switch.

In [ ]:
mac="00:0C:29:47:7A:A1"
!python $base_path/tools/volatility/vol.py --plugins=$plugins_path/AdamBridge -f $case_path/$filename --profile=$profile ndispktscan $kdgb --mac=$mac  --pcap $dump_path/packets2.pcap --dsts $dump_path/ips2.txt
ips2=!cat $dump_path/ips2.txt
#modificar
if "file" in ips2:
    print("Nothing found.")
else:
    for ip in ips2:
        ips_list.append(ip)

<a id='windows_events'></a>
### 10. Windows Event Logs Extraction
[Top](#table_of_contents)

<b>evtlogs:</b> Extract Windows Event Logs (XP/2003 only).

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile evtlogs --dump-dir=$dump_path $kdgb

<b>evtxtract:</b> Extract Windows Event Logs (Vista/7/8/10 only).

In [ ]:
!evtxtract $case_path/$filename

<a id='malicious_code'></a>
### 11. Malicious Code Investigation
[Top](#table_of_contents)

<b>yarascan</b>

In [ ]:
#!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile yarascan

<b>malfind:</b> The malfind command helps find hidden or injected code/DLLs in user mode memory, based on characteristics such as VAD tag and page permissions.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile malfind $kdgb

<b>ldrmodules:</b> There are many ways to hide a DLL. One of the ways involves unlinking the DLL from one (or all) of the linked lists in the PEB. However, when this is done, there is still information contained within the VAD (Virtual Address Descriptor) which identifies the base address of the DLL and its full path on disk. To cross-reference this information (known as memory mapped files) with the 3 PEB lists, use the ldrmodules command.

For each memory mapped PE file, the ldrmodules command prints True or False if the PE exists in the PEB lists. 

Since the PEB and the DLL lists that it contains all exist in user mode, its also possible for malware to hide (or obscure) a DLL by simply overwriting the path. Tools that only look for unlinked entries may miss the fact that malware could overwrite C:\bad.dll to show C:\windows\system32\kernel32.dll. So you can also pass -v or --verbose to ldrmodules to see the full path of all entries.

For concrete examples, see http://blogs.mcafee.com/mcafee-labs/zeroaccess-misleads-memory-file-link ZeroAccess Misleads Memory-File Link and QuickPost: Flame & Volatility.

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile ldrmodules $kdgb

Cobalt strike
https://github.com/RomanEmelyanov/CobaltStrikeForensic

https://blogs.jpcert.or.jp/en/2018/08/volatility-plugin-for-detecting-cobalt-strike-beacon.html

<a id='advanced_extract_files'></a>
### 12. Advanced Investigation - Extracting Files
[Top](#table_of_contents)

<b>dumpfiles</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --dump-dir=$dump_path dumpfiles $kdgb

<b>bulk_extractor:</b>

In [ ]:
!bulk_extractor -o /home/mmatari/Notebooks/investigation_files/ $case_path/$filename
ips_list = []
#ips
ips=[]
ips=!cat $dump_path/ip_histogram.txt
for ip in ips[5:]:
    ip=ip.split('\t')
    ips_list.append(ip[1])
ips_list=set(ips_list)
ips_list=list(ips_list)
print(ips_list)
print()
#urls
urls=[]
urls_list=[]
urls=!cat $dump_path/url_histogram.txt
for url in urls[5:]:
    url=url.split('\t')
    urls_list.append(url[1])
urls_list=set(urls_list)
urls_list=list(urls_list)
print(urls_list)
print()
#packet_pcap
pcap_file=!echo $dump_path"/packets.pcap"
print(pcap_file)

<b>dump file by offset</b>

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output

offset = widgets.Text(description="Offset:", width=200)
display(offset)
def handle_submit(sender):
    clear_output()
    display(strings)
    !python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --offset=$offset.value --dump-dir=$dump_path dumpfiles 
offset.on_submit(handle_submit)

<b>memdump</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --dump-dir=$dump_path memdump $kdgb

<b>procdump</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --dump-dir=$dump_path procdump $kdgb

<b>dlldump</b>

In [ ]:
!python $base_path/tools/volatility/vol.py -f $case_path/$filename --profile=$profile --dump-dir=$dump_path dlldump $kdgb

<a id='advanced_search_strings'></a>
### 13. Advanced Investigation - Searching Strings
[Top](#table_of_contents)

<b>Search strings:</b>

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output

strings = widgets.Text(description="Search:", width=200)
display(strings)
def handle_submit(sender):
    test=!egrep -ai $strings.value $case_path/$filename
    clear_output()
    display(strings)
    for line in test:
        print(line)
strings.on_submit(handle_submit)

<a id='iocs_list'></a>
### 14. IOCs List
[Top](#table_of_contents)

Use this information with the rest of the notebooks in order to investigate it.

In [ ]:
print("IP list:")
print(ips_list)
print("URL list:")
print(urls_list)
print("Files folder:")
print(dump_path)
print("MD5 hashes list:")
print("[TBD]")
pcaps_list=[]
pcapfiles=!ls $dump_path | grep pcap
print("Pcap files:")
for pcap in pcapfiles:
    pcaps_list.append(dump_path+pcap)
print(pcaps_list)